In [37]:
import qiskit
import math
from qiskit.tools.visualization import *
from qiskit.tools.monitor import job_monitor
from qiskit.utils import QuantumInstance
from qiskit.algorithms import Shor
from qiskit import Aer
from qiskit import IBMQ
import qiskit.tools.jupyter
import tqdm
from qiskit import execute

In [2]:
%qiskit_version_table

In [3]:
num2factorize=21

In [4]:
factors = Shor(num2factorize)

In [5]:
simulator = Aer.get_backend('aer_simulator') #qasm_simulator
sim_job = QuantumInstance(backend=simulator,shots=128, skip_qobj_validation = False)

In [6]:
sim_algo = Shor(quantum_instance=sim_job)
type(sim_algo)

qiskit.algorithms.factorizers.shor.Shor

In [7]:
sim_result = sim_algo.factor(num2factorize)

/Users/karim/.pyenv/versions/3.10.2/lib/python3.10/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/karim/.pyenv/versions/3.10.2/lib/python3.10/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


In [9]:
print(f"The list of factors of {num2factorize} as computed by the Shor's algorithm is {sim_result.factors[0]}.")

The list of factors of 21 as computed by the Shor's algorithm is [3, 7].


In [12]:
print(f'The number of qubits in the circuit: {sim_algo.construct_circuit(num2factorize).num_qubits}')

The number of qubits in the circuit: 22


Lets check if we can do the same on IBMQ

In [13]:
with open('/Users/karim/staff/cred/IBMQ/tocken.txt', 'r') as file:
    myTocken = file.read().replace('\n', '')
IBMQ.save_account(myTocken,overwrite=True)
IBMQ.load_account()
IBMQ.providers()

[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>]

get a list of the quantum computing instances that we can run on it
you can find more here: https://quantum-computing.ibm.com/lab/docs/iql/manage/account/ibmq#backends 

In [14]:
provider = IBMQ.get_provider('ibm-q')

In [16]:
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_bogota') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_extended_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_m

In [15]:
provider.backends(simulator=False, operational=True)

[<IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_bogota') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_manila') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [17]:
from qiskit.providers.ibmq import least_busy

In [26]:
small_devices = provider.backends(filters=lambda x: x.configuration().n_qubits < 5 and not x.configuration().simulator)
least_busy(small_devices)

<IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>

In [23]:
big_devices = provider.backends(filters=lambda x: x.configuration().n_qubits >= 5 and not x.configuration().simulator)
big_devices

[<IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_bogota') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_manila') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [24]:
backend = least_busy(big_devices)
backend

<IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q', group='open', project='main')>

In [32]:
backend.configuration().n_qubits

5

In [30]:
quantum_instance = QuantumInstance(backend=least_busy(big_devices),shots=128, skip_qobj_validation = False)

In [31]:
quantum_instance.backend.status()